In [ ]:

from keras.applications import VGG16


# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows = 224
img_cols = 224 
      
#Loads the VGG
model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))


C:\Users\PRASAD\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.
C:\Users\PRASAD\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\PRASAD\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\PRASAD\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of 


26968064/58889256 [============>.................] - ETA: 15:19

# Making a Face recognizer with VGG16

### Loading the VGG16 Model

### Inpsecting each layer

In [ ]:
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

### Let's freeze all layers except the top 4 

In [ ]:
from keras.applications import VGG16

# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows = 224
img_cols = 224 

# Re-loads the VGG16 model without the top or FC layers
model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default

for layer in model.layers:
    layer.trainable = False #your said images are not of 
    
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

### Let's make a function that returns our FC Head

In [ ]:
def addTopModel(bottom_model, num_classes, D=256):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""
    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    #top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model

In [ ]:
model.input

In [ ]:
model.layers

### Let's add our FC Head back onto VGG

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 2

FC_Head = addTopModel(model, num_classes)

modelnew = Model(inputs=model.input, outputs=FC_Head)

print(modelnew.summary())

In [ ]:
#

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
 
train_data_dir = 'Trainingset/train/'
validation_data_dir = 'Trainingset/val/'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True, 
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

In [ ]:
from tensorflow.python import keras
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
modelnew.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics = ['accuracy'])

nb_train_samples = 14

nb_validation_samples = 11
epochs = 3
batch_size = 16

history = modelnew.fit_generator(
    train_generator,
    steps_per_epoch = 10,
    epochs = epochs,
    verbose=1,
    validation_data = validation_generator,
    validation_steps = 8)

model.save("classfy_person.h5")

In [ ]:
from tensorflow.keras.models import load_model
from keras.layers import Flatten
#model.Flatten()
classifier = load_model('classify_person.h5')


In [ ]:
from keras.preprocessing import image
import numpy as np     
test_image = image.load_img("Trainingset/val/Family_photo/img241019.jpg",target_size=(224,224))
path = "Trainingset/val/Family_photo/img241019.jpg"

In [ ]:
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image.shape

In [ ]:
result = classifier.predict(test_image)
print(result)
#print(classifier.predict(test_image))

In [ ]:

def show_img(name,path_f):
    import cv2
    
    imgshow = cv2.imread(path_f) 
    font = cv2.FONT_HERSHEY_SIMPLEX 

    # org
    org = (25, 25) 

    # fontScale 
    fontScale = 0.5

    # Blue color in BGR 
    color = (255, 0, 0) 

    # Line thickness of 2 px 
    thickness = 1

    # Using cv2.putText() method 
    image_f = cv2.putText(imgshow, name, org, font,  
                       fontScale, color, thickness, cv2.LINE_AA) 

    # Displaying the image 
    cv2.imshow('test', image_f)  
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
print(result[0][0][0])

In [1]:
def pred_m(result,path):
    imgs = '0'
    
    if result[0][0] >= 1:
        imgs = 'Prasad Joshi'
        print(path)
        show_img(imgs ,path) 
    elif result[0][1] >= 1:
        imgs = 'Mukund Joshi'
        show_img(imgs ,path) 
    else:
        imgs = "Unidentified person"
        show_img(imgs,path)


In [ ]:
pred_m(result,"Trainimg/family/img120518.jpg")